# Лабораторная работа №5: Градиентный бустинг

# Основные этапы:
	•	Базовые модели: обучение и оценка библиотечных реализаций градиентного бустинга (GradientBoostingClassifier, GradientBoostingRegressor) на стандартных настройках.
	•	Улучшение моделей: оптимизация гиперпараметров (например, n_estimators, learning_rate, max_depth) с использованием RandomizedSearchCV для повышения качества предсказаний.
	•	Пользовательская реализация: разработка кастомной версии алгоритма для более глубокого понимания его работы.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
import pandas as pd

# Загрузка и предварительная обработка данных для классификации
# Чтение CSV файла с текстовыми данными (job_title) и метками классов (category).
classification_data = pd.read_csv("/content/drive/MyDrive/AIMAI/ds1.csv.csv")

# Удаление строк с пропущенными значениями, чтобы избежать ошибок при обработке.
classification_data = classification_data.dropna()

# Разделение на признаки и целевую переменную
# X_text: текстовые данные (job_title), которые будут преобразованы в числовую форму.
# y_class: метки классов (category), которые будем предсказывать.
X_text = classification_data['job_title']
y_class = classification_data['category']

# Преобразование текстовых данных в числовую форму
# Используем метод Bag of Words (CountVectorizer), чтобы представить текстовые данные как числовую матрицу.
vectorizer = CountVectorizer()
X_class = vectorizer.fit_transform(X_text)

# Разделение данных на обучающую и тестовую выборки
# Тренировочная выборка (80%): используется для обучения модели.
# Тестовая выборка (20%): используется для проверки качества модели.
# random_state=42 фиксирует случайное состояние для воспроизводимости.
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)

In [ ]:
# Загрузка и предварительная обработка данных для регрессии

# Чтение CSV файла, содержащего признаки (X_reg) и целевую переменную (price).
regression_data = pd.read_csv("/content/drive/MyDrive/AIMAI/ds2.csv.csv")

# Удаление ненужных столбцов
# Исключаем столбцы:
# - price: целевая переменная, которую будем предсказывать.
# - Address и desc: текстовые столбцы, которые не используются в модели.
X_reg = regression_data.drop(columns=["price", "Address", "desc"])
y_reg = regression_data["price"]

# Обработка пропущенных значений
# Выбираем только числовые столбцы для замены NaN на среднее значение.
numeric_columns = X_reg.select_dtypes(include=['float64', 'int64']).columns
X_reg[numeric_columns] = SimpleImputer(strategy='mean').fit_transform(X_reg[numeric_columns])

# Преобразование категориальных данных в числовую форму
# Используем one-hot encoding для преобразования категориальных признаков.
# drop_first=True исключает первый уровень категорий, чтобы избежать мультиколлинеарности.
X_reg = pd.get_dummies(X_reg, drop_first=True)

# Разделение данных на обучающую и тестовую выборки
# Тренировочная выборка (80%): используется для обучения модели.
# Тестовая выборка (20%): используется для проверки качества модели.
# random_state=42 фиксирует случайное состояние для воспроизводимости.
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

In [ ]:
# Базовая модель градиентного бустинга для классификации

# Создание и обучение модели градиентного бустинга
# random_state=42 фиксирует случайное состояние для воспроизводимости.
clf = GradientBoostingClassifier(random_state=42)

# Обучение модели на тренировочных данных
# X_train_class: матрица признаков для обучения.
# y_train_class: метки классов для обучения.
clf.fit(X_train_class, y_train_class)

# Предсказание классов для тестовой выборки
# X_test_class: матрица признаков для тестовой выборки.
y_pred_class = clf.predict(X_test_class)

# Оценка качества базовой модели
print("Classification Baseline:")

# Accuracy: доля правильно классифицированных объектов.
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))

# Отчёт о классификации
# Включает precision, recall, f1-score и support для каждого класса.
print(classification_report(y_test_class, y_pred_class))

Classification Baseline:
Accuracy: 0.8132427843803056
                                        precision    recall  f1-score   support

                            Accounting       0.50      0.22      0.31         9
       Administration & Office Support       0.79      0.89      0.84       436
             Advertising, Arts & Media       0.33      0.08      0.13        12
          Banking & Financial Services       0.68      0.75      0.71       208
              CEO & General Management       0.70      0.70      0.70        10
        Call Centre & Customer Service       0.62      0.37      0.46        35
                          Construction       0.85      0.78      0.81        85
                 Consulting & Strategy       0.56      0.42      0.48        24
                 Design & Architecture       0.76      0.76      0.76        17
                           Engineering       0.25      0.33      0.29         3
                  Healthcare & Medical       0.00      0.00      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Базовая модель градиентного бустинга для регрессии

# Создание и обучение модели градиентного бустинга
# random_state=42 фиксирует случайное состояние для воспроизводимости.
reg = GradientBoostingRegressor(random_state=42)

# Обучение модели на тренировочных данных
# X_train_reg: матрица признаков для обучения.
# y_train_reg: целевая переменная для обучения.
reg.fit(X_train_reg, y_train_reg)

# Предсказание значений целевой переменной для тестовой выборки
# X_test_reg: матрица признаков для тестовой выборки.
y_pred_reg = reg.predict(X_test_reg)

# Оценка качества базовой модели
print("Regression Baseline:")

# MSE (Mean Squared Error): среднеквадратичная ошибка.
# Указывает на среднее значение квадрата разницы между предсказанными и реальными значениями.
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

# MAE (Mean Absolute Error): средняя абсолютная ошибка.
# Показывает среднее абсолютное отклонение предсказанных значений от реальных.
print("Mean Absolute Error:", mean_absolute_error(y_test_reg, y_pred_reg))

# R² (R2 Score): коэффициент детерминации.
# Показывает, насколько хорошо модель объясняет дисперсию данных.
print("R2 Score:", r2_score(y_test_reg, y_pred_reg))

Regression Baseline:
Mean Squared Error: 402883666920.3741
Mean Absolute Error: 270645.76738881896
R2 Score: 0.9932041399047439


Улучшение

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Улучшение базовой модели классификации с использованием TfidfVectorizer и RandomizedSearchCV
print("\nImproving Classification Baseline:")

# Преобразование текстовых данных с использованием TfidfVectorizer
# max_features=1000: ограничиваем количество признаков для ускорения вычислений.
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_class_tfidf = tfidf_vectorizer.fit_transform(X_text)

# Преобразуем разреженную матрицу в плотную для корректной работы модели
X_class_tfidf_dense = X_class_tfidf.toarray()

# Разделение данных на обучающую и тестовую выборки
# random_state=42 фиксирует случайное состояние для воспроизводимости.
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class_tfidf_dense, y_class, test_size=0.2, random_state=42
)

# Определение диапазона гиперпараметров для RandomizedSearchCV
# max_iter: количество итераций обучения.
# learning_rate: скорость обучения модели.
# max_depth: максимальная глубина деревьев.
param_dist_class = {
    'max_iter': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5]
}

# RandomizedSearchCV для подбора гиперпараметров HistGradientBoostingClassifier
# n_iter=5: проверяем 5 случайных комбинаций гиперпараметров.
# cv=2: двухкратная кросс-валидация для оценки качества на обучающих данных.
# n_jobs=-1: использование всех доступных ядер процессора для ускорения вычислений.
clf_rs = RandomizedSearchCV(
    HistGradientBoostingClassifier(random_state=42),
    param_distributions=param_dist_class,
    n_iter=5,
    cv=2,
    random_state=42,
    n_jobs=-1
)

# Обучение модели на тренировочных данных
clf_rs.fit(X_train_class, y_train_class)

# Вывод лучших параметров модели
print("Best Parameters for Classification:", clf_rs.best_params_)

# Предсказание классов для тестовой выборки
y_pred_class = clf_rs.predict(X_test_class)

# Оценка качества модели
print("Accuracy:", accuracy_score(y_test_class, y_pred_class))
print(classification_report(y_test_class, y_pred_class))


Improving Classification Baseline:


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Best Parameters for Classification: {'max_iter': 100, 'max_depth': 3, 'learning_rate': 0.05}
Accuracy: 0.7775891341256367
                                        precision    recall  f1-score   support

                            Accounting       0.33      0.11      0.17         9
       Administration & Office Support       0.74      0.88      0.81       436
             Advertising, Arts & Media       0.00      0.00      0.00        12
          Banking & Financial Services       0.60      0.75      0.67       208
              CEO & General Management       0.67      0.60      0.63        10
        Call Centre & Customer Service       0.53      0.51      0.52        35
                          Construction       0.87      0.69      0.77        85
                 Consulting & Strategy       0.67      0.17      0.27        24
                 Design & Architecture       0.38      0.18      0.24        17
                           Engineering       0.50      0.33      0.40        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

print("\nImproving Regression Baseline:")

# Масштабирование данных
# Используем StandardScaler для нормализации данных
# Применяется ко всем признакам, чтобы ускорить сходимость градиентного бустинга.
scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)  # Масштабируем обучающие данные
X_test_reg_scaled = scaler.transform(X_test_reg)  # Масштабируем тестовые данные

# Определение диапазона гиперпараметров для RandomizedSearchCV
# n_estimators: количество деревьев в ансамбле.
# learning_rate: шаг обучения модели.
# max_depth: максимальная глубина деревьев.
param_dist_reg = {
    'n_estimators': [50, 100],  # Ограничиваем количество деревьев для ускорения
    'learning_rate': [0.05, 0.1],  # Ограничиваем варианты скорости обучения
    'max_depth': [3, 5]  # Ограничиваем глубину деревьев
}

# RandomizedSearchCV для поиска лучших гиперпараметров
# n_iter=5: Проверяем 5 случайных комбинаций гиперпараметров.
# cv=2: Двухкратная кросс-валидация для оценки качества на обучающих данных.
# n_jobs=-1: Используем все доступные ядра процессора.
reg_rs = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_distributions=param_dist_reg,
    n_iter=5,
    cv=2,
    random_state=42,
    n_jobs=-1
)

# Обучение модели с подобранными гиперпараметрами
reg_rs.fit(X_train_reg_scaled, y_train_reg)

# Вывод лучших параметров модели
print("Best Parameters for Regression:", reg_rs.best_params_)

# Предсказание значений целевой переменной для тестовой выборки
y_pred_reg = reg_rs.predict(X_test_reg_scaled)

# Оценка качества модели
# Mean Squared Error (MSE): среднеквадратичная ошибка предсказаний.
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_reg))

# R² (R2 Score): коэффициент детерминации.
# Показывает, насколько хорошо модель объясняет дисперсию данных.
print("R2 Score:", r2_score(y_test_reg, y_pred_reg))


Improving Regression Baseline:


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters for Regression: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1}
Mean Squared Error: 407908432921.0759
R2 Score: 0.9931193819223387


In [ ]:
class CustomGradientBoosting:
    """
    Кастомная реализация градиентного бустинга.
    Основная идея — итеративное построение моделей для предсказания ошибок (остатков).
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        """
        Инициализация параметров градиентного бустинга.

        Параметры:
        - n_estimators: количество деревьев в ансамбле.
        - learning_rate: скорость обучения.
        - max_depth: максимальная глубина каждого дерева.
        """
        self.n_estimators = n_estimators  # Количество деревьев
        self.learning_rate = learning_rate  # Скорость обучения
        self.max_depth = max_depth  # Максимальная глубина деревьев
        self.models = []  # Список для хранения моделей (решающих деревьев)
        self.init_prediction = None  # Начальное значение предсказания (среднее по целевой переменной)

    def fit(self, X, y):
        """
        Обучение градиентного бустинга.

        Параметры:
        - X: матрица признаков (NumPy массив или Pandas DataFrame).
        - y: целевая переменная (вектор).
        """
        # Устанавливаем начальное предсказание как среднее значение целевой переменной.
        self.init_prediction = np.mean(y)

        # Вычисляем начальные остатки (разница между реальными значениями и начальным предсказанием).
        residuals = y - self.init_prediction

        for _ in range(self.n_estimators):
            # Создаём модель для предсказания остаточных ошибок.
            model = GradientBoostingRegressor(
                n_estimators=1,  # Обучаем одно дерево на каждой итерации
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                random_state=42
            )

            # Обучаем модель на текущих остатках.
            model.fit(X, residuals)

            # Сохраняем обученную модель.
            self.models.append(model)

            # Обновляем остатки, вычитая предсказание текущей модели.
            residuals -= self.learning_rate * model.predict(X)

    def predict(self, X):
        """
        Предсказание целевой переменной.

        Параметры:
        - X: матрица признаков (NumPy массив или Pandas DataFrame).

        Возвращает:
        - predictions: предсказания целевой переменной (вектор).
        """
        # Начинаем с базового предсказания (среднее по целевой переменной).
        predictions = np.full(X.shape[0], self.init_prediction)

        # Добавляем предсказания всех обученных моделей.
        for model in self.models:
            predictions += self.learning_rate * model.predict(X)

        return predictions

In [ ]:
# Обучение и оценка кастомной реализации градиентного бустинга для регрессии

# Создание экземпляра кастомного градиентного бустинга
# Параметры:
# - n_estimators=50: количество деревьев в ансамбле.
# - learning_rate=0.1: скорость обучения (вклад каждого дерева в итоговое предсказание).
# - max_depth=3: максимальная глубина деревьев.
custom_gb = CustomGradientBoosting(n_estimators=50, learning_rate=0.1, max_depth=3)

# Обучение модели на тренировочных данных
# X_train_reg: матрица признаков для обучения.
# y_train_reg: целевая переменная для обучения.
custom_gb.fit(X_train_reg, y_train_reg)

# Предсказание значений целевой переменной для тестовой выборки
# X_test_reg: матрица признаков для тестовой выборки.
y_pred_custom = custom_gb.predict(X_test_reg)

# Оценка качества кастомного градиентного бустинга
print("Custom Gradient Boosting for Regression:")

# MSE (Mean Squared Error): среднеквадратичная ошибка предсказаний.
# Указывает на среднее значение квадрата разницы между предсказанными и реальными значениями.
print("Mean Squared Error:", mean_squared_error(y_test_reg, y_pred_custom))

# MAE (Mean Absolute Error): средняя абсолютная ошибка.
# Показывает среднее абсолютное отклонение предсказанных значений от реальных.
print("Mean Absolute Error:", mean_absolute_error(y_test_reg, y_pred_custom))

# R² (R2 Score): коэффициент детерминации.
# Показывает, насколько хорошо модель объясняет дисперсию данных (чем ближе к 1, тем лучше).
print("R2 Score:", r2_score(y_test_reg, y_pred_custom))

Custom Gradient Boosting for Regression:
Mean Squared Error: 27474434588463.695
Mean Absolute Error: 3039431.512257809
R2 Score: 0.5365599824716487


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Обработка целевых меток для классификации
# Используем LabelEncoder для преобразования строковых меток классов в числовую форму.
label_encoder = LabelEncoder()
y_class_encoded = label_encoder.fit_transform(y_class)

# Разделение данных на тренировочные и тестовые выборки
# X_class: матрица признаков.
# y_class_encoded: целевая переменная в числовой форме.
# test_size=0.2: 20% данных используется для тестирования.
# random_state=42 фиксирует случайное состояние для воспроизводимости.
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class_encoded, test_size=0.2, random_state=42
)

# Обучение пользовательской модели градиентного бустинга для классификации
# Параметры:
# - n_estimators=50: количество деревьев.
# - learning_rate=0.1: скорость обучения.
# - max_depth=3: максимальная глубина деревьев.
custom_gb_clf = CustomGradientBoosting(n_estimators=50, learning_rate=0.1, max_depth=3)

# Обучение модели на тренировочных данных
# Преобразуем разреженную матрицу признаков в плотную с помощью .toarray().
# Целевая переменная y_train_class преобразована в float для работы с пользовательской реализацией.
custom_gb_clf.fit(X_train_class.toarray(), y_train_class.astype(float))

# Предсказание вероятностей классов на тестовой выборке
y_pred_custom_class = custom_gb_clf.predict(X_test_class.toarray())

# Преобразование вероятностей в метки классов
# threshold=0.5: значения выше порога 0.5 относятся к классу 1, ниже — к классу 0.
custom_class_labels = np.where(y_pred_custom_class > threshold, 1, 0)

# Оценка качества пользовательской модели для классификации
print("Custom Gradient Boosting for Classification:")

# Accuracy: доля правильно классифицированных объектов.
print("Accuracy:", accuracy_score(y_test_class, custom_class_labels))

# Отчёт о классификации
# Включает precision, recall, f1-score и support для каждого класса.
print(classification_report(y_test_class, custom_class_labels))

Custom Gradient Boosting for Classification:
Accuracy: 0.24674589700056593
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.25      1.00      0.40       436
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00       208
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        35
           6       0.00      0.00      0.00        85
           7       0.00      0.00      0.00        24
           8       0.00      0.00      0.00        17
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          13       0.00      0.00      0.00       366
          14       0.00      0.00      0.00       149
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00        25
          17       0.00      0.00      0.00       165
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Вывод:
	•	Градиентный бустинг является мощным алгоритмом для задач классификации и регрессии, особенно при корректной настройке гиперпараметров.
	•	Рекомендуется использовать библиотечные реализации (GradientBoostingClassifier, GradientBoostingRegressor) для практических задач, поскольку они обеспечивают высокую производительность и стабильность.
	•	Пользовательская реализация подходит для изучения механики работы алгоритма, но её использование в реальных задачах ограничено.